# Lab 2 - Automatic Creation of `sqlalchemy` data type `dict`

So far, we have been manually constructing the `sqlalchemy` type `dict`, but this approach quickly becomes unwieldy.  Let's look at using the `pandas` types to programmically construct the type `dict`.

In [1]:
import pandas as pd
from dfply import *

In [28]:
!rm databases/baseball.db

## Case Study - People

Let's use the `People.csv` file from the [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/) as our motivating example, since 

1. It has examples of a number of types
2. It has lots of columns and would be annoying to manually construct the type `dict`.

In [3]:
people = pd.read_csv('~/Desktop/baseball/core/People.csv')
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [10]:
people.shape

(19370, 24)

## <font color="red"> Exercise 1 </font>

Inspect the types of the `people` table and make note of any necessary changes.

In [11]:
# Your code here

*Your thoughts here*

## Missing `Int64` columns

As mentioned in [Lecture 1/4](./pbpython/notebooks/1_4_more_on_pandas_data_types_key.ipynb), we need to use the most recent version `pandas` (still in development as of this writing) to allow us to have integer columns with missing values.

In [13]:
assert pd.__version__.startswith('0.24'), "Please uncomment and run the pip command to upgrade pandas"
#!pip install --upgrade --pre pandas

## Correcting the `pandas` types

1. We pass `parse_dates` a list of date columns
2. We pass `dtypes` a `dict` of types of the birth and death columns

#### Constructing the `dtype` `dict`

In [23]:
date_cols = ['debut', 'finalGame']
birth_death_date_cols = [prefix + time for prefix in ('birth', 'death') for time in ('Year', 'Month', 'Day')]
people_dtypes = {col:pd.Int64Dtype() for col in people.columns if col in birth_death_date_cols}
people_dtypes

{'birthYear': Int64Dtype(),
 'birthMonth': Int64Dtype(),
 'birthDay': Int64Dtype(),
 'deathYear': Int64Dtype(),
 'deathMonth': Int64Dtype(),
 'deathDay': Int64Dtype()}

## Rereading the csv with the correct types

In [25]:
people = pd.read_csv('~/Desktop/baseball/core/People.csv', dtype=people_dtypes, parse_dates=date_cols)
people.dtypes

playerID                object
birthYear                Int64
birthMonth               Int64
birthDay                 Int64
birthCountry            object
birthState              object
birthCity               object
deathYear                Int64
deathMonth               Int64
deathDay                 Int64
deathCountry            object
deathState              object
deathCity               object
nameFirst               object
nameLast                object
nameGiven               object
weight                 float64
height                 float64
bats                    object
throws                  object
debut           datetime64[ns]
finalGame       datetime64[ns]
retroID                 object
bbrefID                 object
dtype: object

## <font color="red"> Exercise 2 </font>
**Goal:** Find a method/attribute of each `dtype`, which preferably returns something immutable like a `str`, that we can use to identify the general type.
**Tasks:**

1. Pull off an example `dtype`
2. Use `dir` to inspect the available methods
3. Test the methods/attributes to find a good candidate.

In [28]:
# Your code here

## Creating a type conversion dictionary

1. keys will be the `dtype.kind` strings
2. The `values` will be the associated `sqlalchemy` types

In [34]:
from sqlalchemy import Integer, Float, String, DateTime
DTYPES_TO_SQLALCHEMY_TYPES = {'O':String,
                              'i':Integer,
                              'f':Float,
                              'M':DateTime}
DTYPES_TO_SQLALCHEMY_TYPES

{'O': sqlalchemy.sql.sqltypes.String,
 'i': sqlalchemy.sql.sqltypes.Integer,
 'f': sqlalchemy.sql.sqltypes.Float,
 'M': sqlalchemy.sql.sqltypes.DateTime}

## Use ALL CAPS for global constants

When dealing with global constants, we should

1. Define them at the top of the file.
2. Use an ALL CAPS name to make them stand out.

## <font color="red"> Exercise 3 </font>

Write a `dict` comprehension that uses our conversion `dict` to convert the `pandas` `dtypes` to `sqlalchemy` types.

In [35]:
# Your code here

## <font color="red"> Exercise 4 </font>

Package your expression in a `lambda` and refactor your code by adding helper functions to clean up the expression.

In [ ]:
# lambda function before refactoring

In [37]:
# refactored lambda function and helper functions

## <font color="red"> Exercise 5 </font>

Add the `People.csv` to your `baseball.db`

In [ ]:
# lambda function before refactoring

In [1]:
# refactored lambda function and helper functions

## <font color="red"> Exercise 6 </font>

Set up a similar automatic conversion precess for `pyspark.DataFrames`. You need to build a structure like this:

```python
schema = StructType([StructField('Name', StringType(), True),
                     StructField('DateTime', TimestampType(), True)
                     StructField('Age', IntegerType(), True)])
```